<a href="https://colab.research.google.com/github/chuchun2/ATMS597-Project6-GroupB/blob/master/ATMS597_Project6_GroupB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%pylab inline
import xarray as xr
import pandas as pd
import glob
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from math import sqrt
from google.colab import drive
drive.mount('/content/drive')

Populating the interactive namespace from numpy and matplotlib
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# Set your directory path
#YOUR_DIRECTORY = '/content/drive/My Drive/ATMS597 Weather Climate Data Analysis/Module 6/Project 6/'  # Chu-Chun
YOUR_DIRECTORY = '/content/drive/My Drive/Colab Notebooks/Project_6/' # Jeff

In [0]:
# Load MJO data
MJOIndices_df = pd.read_csv(YOUR_DIRECTORY + 'MJOIndices_JMA_1980to2020.csv', index_col = 'Unnamed: 0')
MJOIndices_df.columns.name = 'Date'
MJOIndices_df

Date,MJOPhase_OLR,MJOLongitude_OLR,MJOAmplitude_OLR,MJOPhase_Chi200,MJOLongitude_Chi200,MJOAmplitude_Chi200
1980-01-01,5,148.6,0.6049,5,175.4,0.5459
1980-01-02,5,164.0,0.6069,6,187.0,0.7355
1980-01-03,5,176.0,0.5613,6,198.4,0.6891
1980-01-04,6,185.2,0.6842,6,191.3,0.7448
1980-01-05,6,199.9,0.8435,6,215.7,0.8425
...,...,...,...,...,...,...
2020-02-15,5,176.4,1.6450,5,180.0,1.6147
2020-02-16,5,171.4,1.7534,5,167.1,1.7944
2020-02-17,5,164.4,1.7153,5,166.2,1.9357
2020-02-18,5,163.8,1.6732,5,172.0,1.8460


In [0]:
# Dropping unneeded columns
MJOIndices_df = MJOIndices_df.drop(columns = ['MJOLongitude_OLR', 'MJOPhase_Chi200', 'MJOLongitude_Chi200', 'MJOAmplitude_Chi200'])
MJOIndices_df

Date,MJOPhase_OLR,MJOAmplitude_OLR
1980-01-01,5,0.6049
1980-01-02,5,0.6069
1980-01-03,5,0.5613
1980-01-04,6,0.6842
1980-01-05,6,0.8435
...,...,...
2020-02-15,5,1.6450
2020-02-16,5,1.7534
2020-02-17,5,1.7153
2020-02-18,5,1.6732


In [0]:
# Download GPCP Daily data

# Data originally collected by Arka Mitra for Project 3 [NOTE: directory no longer active]
years = np.arange(1996, 2020)
gpcp_daily_data_directory = "/content/drive/My Drive/noaa_data_project3/data/global-precipitation-climatology-project-gpcp-daily/access/"

#Calculate total number of days in dataset
count = 0
times = []
for i in range(0, len(years)): 
  nc_files = sorted(glob.glob(gpcp_daily_data_directory + str(years[i]) + "/*.nc"))
  for n in range(0, len(nc_files)):
    filename = nc_files[n].replace(gpcp_daily_data_directory+str(years[i]), '')
    date = filename.split('_')[3].replace('d', '')
    date = pd.to_datetime(date, format = '%Y%m%d')
    times = np.append(times, date)

count = 0
times = []
for i in range(0, len(years)): 
  nc_files = sorted(glob.glob(gpcp_daily_data_directory + str(years[i]) + "/*.nc"))
  for n in range(0, len(nc_files)):
    filename = nc_files[n].replace(gpcp_daily_data_directory + str(years[i]), '')
    date = filename.split('_')[3].replace('d', '')
    date = pd.to_datetime(date, format = '%Y%m%d')
    times = np.append(times, date)
    
latitude = np.arange(-90.0, 90.0)
longitude = np.arange(0.0, 360.0)
data = np.zeros((len(times), len(latitude), len(longitude), 1))

count = 0 #loop over years and store daily data into 'data' array
for i in range(0, len(years)): 
  #print(i)
  nc_files = glob.glob(gpcp_daily_data_directory + str(years[i]) + "/*.nc")
  for n in range(0, len(nc_files)):
    try:
      nc = xr.open_dataset(nc_files[n])
      ncvar = nc['precip']
      precip = ncvar.mean(axis = 0)
      data[count, :, :, 0] = precip
    except:
      continue
    count = count + 1

data1 = data.squeeze(axis = 3)
precip_agg = xr.DataArray(data1, coords = [times, latitude, longitude], dims = ['time', 'latitude', 'longitude'])
#precip_agg.to_netcdf('/content/drive/My Drive/Aggregate_GCPC_daily_1996_2019.nc') #Output NetCDF file generated and saved

In [0]:
# GPCP daily data averaged over SESA

import pandas as pd
import datetime as dt
import numpy as np
from IPython import display # for updating the cell dynamically
import xarray as xr
import glob
!pip install netcdf4

# GPCP Data file from Project 3
Precip_All = xr.open_dataset("/content/drive/My Drive/Colab Notebooks/Project 3/Aggregate_GCPC_daily_1996_2019.nc")

# Averaging GPCP data over SESA
lats = np.array(np.linspace(-25.0, -30.0, 6)) #25-30S
lons = np.array(np.linspace(305.0, 310.0, 6)) #50-55W
precip = Precip_All.sel(latitude = lats, longitude = lons)
precip = precip.where(precip != -99999.0)
precip = precip.where(precip >= 0.0)
precip = precip.where(precip <= 500.0)
precip_mean = np.nanmean(precip['__xarray_dataarray_variable__'], axis = (1, 2)) #7 NaN values out of 8507 points

# Creating GPCP dataframe, which averages over duplicate days from the GPCP online archive
#idx = pd.date_range('1996-10-01', '2019-11-30')
GPCP_PrecipAvg_df = pd.DataFrame(data = precip_mean, index = Precip_All['time'].values, columns = ['GPCP_PrecipAvg'])
GPCP_PrecipAvg_NoDups_df = GPCP_PrecipAvg_df.resample('D').mean()

# Fill in missing days with NaN
idx = pd.date_range('1980-01-01', '2020-02-19')
GPCP_PrecipAvg_NoDups_df = GPCP_PrecipAvg_NoDups_df.reindex(idx, fill_value = NaN)
GPCP_PrecipAvg_NoDups_df.index.name = 'date'
GPCP_PrecipAvg_NoDups_df

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: Mean of empty slice


,GPCP_PrecipAvg
date,
1980-01-01,NaN
1980-01-02,NaN
1980-01-03,NaN
1980-01-04,NaN
1980-01-05,NaN
...,...
2020-02-15,NaN
2020-02-16,NaN
2020-02-17,NaN


In [0]:
## Finding duplicate days in GPCP data

# from collections import Counter
# precip_list = list(precip['time'].values)
# [k for k, v in Counter(precip_list).items() if v > 1]

In [0]:
## For reference from previous iteration when used station precipitation data
# Load Bahia Blanca, AR station data
#AR_data = pd.read_csv(YOUR_DIRECTORY + 'ghcn_data_ar.csv', index_col = 'date')

# Fill in missing days with NaN
#idx = pd.date_range('1980-01-01', '2019-12-31')
#AR_data.index = pd.to_datetime(AR_data['date'])
#AR_data = AR_data.reindex(idx, fill_value = NaN)

# Load Presidente General Don Oscar, UR station data
#UR_data = pd.read_csv(YOUR_DIRECTORY + 'ghcn_data_ur.csv', index_col = 'date')

# # Fill in missing days with NaN
# idx = pd.date_range('1980-01-01', '2019-12-31')
# UR_data.index = pd.to_datetime(UR_data['date'])
# UR_data = UR_data.reindex(idx, fill_value = NaN)

#UR_data

In [0]:
# Load ERA5 soil moisture data
SESA_Avg_data1 = xr.open_dataset(YOUR_DIRECTORY + 'ERA5_swvl1_BR.nc')
SESA_Avg_data2 = xr.open_dataset(YOUR_DIRECTORY + 'ERA5_swvl2_BR.nc')
SESA_Avg_swvl1_aave = np.mean(SESA_Avg_data1['swvl1'], axis=(1, 2))
SESA_Avg_swvl2_aave = np.mean(SESA_Avg_data2['swvl2'], axis=(1, 2))
SESA_Avg_ERA5_soil_moisture1_df = SESA_Avg_swvl1_aave.to_dataframe()
SESA_Avg_ERA5_soil_moisture2_df = SESA_Avg_swvl2_aave.to_dataframe()
SESA_Avg_ERA5_soil_moisture1_df.index = SESA_Avg_ERA5_soil_moisture1_df.index.strftime('%Y-%m-%d')
SESA_Avg_ERA5_soil_moisture2_df.index = SESA_Avg_ERA5_soil_moisture2_df.index.strftime('%Y-%m-%d')

In [0]:
# Make copy of MJO Indices dataframe for each station
MJOIndices_SESA_Avg_df = MJOIndices_df.copy()

# Add GPCP Precipitation data into the dataframe
MJOIndices_SESA_Avg_df['SESA_Avg_GPCP_Precip'] = GPCP_PrecipAvg_NoDups_df['GPCP_PrecipAvg']

# Add ERA5 soil moisture data into the dataframe
MJOIndices_SESA_Avg_df['SESA_Avg_ERA5_Soil_Moisture_Lev1'] = SESA_Avg_ERA5_soil_moisture1_df
MJOIndices_SESA_Avg_df['SESA_Avg_ERA5_Soil_Moisture_Lev2'] = SESA_Avg_ERA5_soil_moisture2_df

# Adding Year/Month/Day, Removing rows for 2020
MJOIndices_SESA_Avg_df.index = pd.to_datetime(MJOIndices_SESA_Avg_df.index)
MJOIndices_SESA_Avg_df['Year'] = MJOIndices_SESA_Avg_df.index.year
MJOIndices_SESA_Avg_df['Month'] = MJOIndices_SESA_Avg_df.index.month
MJOIndices_SESA_Avg_df['Day'] = MJOIndices_SESA_Avg_df.index.day
MJOIndices_SESA_Avg_df = MJOIndices_SESA_Avg_df[MJOIndices_SESA_Avg_df['Year'] != 2020]

# Change column order
MJOIndices_SESA_Avg_df = MJOIndices_SESA_Avg_df[['SESA_Avg_GPCP_Precip', 'MJOPhase_OLR', 'MJOAmplitude_OLR', 'SESA_Avg_ERA5_Soil_Moisture_Lev1', 'SESA_Avg_ERA5_Soil_Moisture_Lev2',
                                                 'Year', 'Month', 'Day']]

# Adding lagged MJO and ERA5 data into the dataframe
SESA_Avg_Variable_Strings = ['MJOPhase_OLR', 'MJOAmplitude_OLR', 'SESA_Avg_ERA5_Soil_Moisture_Lev1', 'SESA_Avg_ERA5_Soil_Moisture_Lev2']
Lag_Days = np.arange(5, 35, 5)

for i in np.arange(0,len(SESA_Avg_Variable_Strings)):
  for j in np.arange(0,len(Lag_Days)):
    MJOIndices_SESA_Avg_df[SESA_Avg_Variable_Strings[i] + '_lag' + str(Lag_Days[j])] = MJOIndices_SESA_Avg_df[SESA_Avg_Variable_Strings[i]].shift(periods = Lag_Days[j])

MJOIndices_SESA_Avg_df = MJOIndices_SESA_Avg_df.dropna()
#MJOIndices_SESA_Avg_df.to_csv(YOUR_DIRECTORY + 'SESA_Avg_Precip_MJOIndices_SoilMoisture.csv')

In [0]:
# Load saved combined data
df = pd.read_csv(YOUR_DIRECTORY + 'SESA_Avg_Precip_MJOIndices_SoilMoisture.csv', index_col = 'Unnamed: 0')
df.columns.name = 'Date'
df

Date,SESA_Avg_GPCP_Precip,MJOPhase_OLR,MJOAmplitude_OLR,SESA_Avg_ERA5_Soil_Moisture_Lev1,SESA_Avg_ERA5_Soil_Moisture_Lev2,Year,Month,Day,MJOPhase_OLR_lag5,MJOPhase_OLR_lag10,MJOPhase_OLR_lag15,MJOPhase_OLR_lag20,MJOPhase_OLR_lag25,MJOPhase_OLR_lag30,MJOAmplitude_OLR_lag5,MJOAmplitude_OLR_lag10,MJOAmplitude_OLR_lag15,MJOAmplitude_OLR_lag20,MJOAmplitude_OLR_lag25,MJOAmplitude_OLR_lag30,SESA_Avg_ERA5_Soil_Moisture_Lev1_lag5,SESA_Avg_ERA5_Soil_Moisture_Lev1_lag10,SESA_Avg_ERA5_Soil_Moisture_Lev1_lag15,SESA_Avg_ERA5_Soil_Moisture_Lev1_lag20,SESA_Avg_ERA5_Soil_Moisture_Lev1_lag25,SESA_Avg_ERA5_Soil_Moisture_Lev1_lag30,SESA_Avg_ERA5_Soil_Moisture_Lev2_lag5,SESA_Avg_ERA5_Soil_Moisture_Lev2_lag10,SESA_Avg_ERA5_Soil_Moisture_Lev2_lag15,SESA_Avg_ERA5_Soil_Moisture_Lev2_lag20,SESA_Avg_ERA5_Soil_Moisture_Lev2_lag25,SESA_Avg_ERA5_Soil_Moisture_Lev2_lag30
1996-10-01,0.802094,4,0.4059,0.460888,0.468150,1996,10,1,6.0,6.0,5.0,5.0,5.0,1.0,0.6244,1.3798,1.0795,1.8319,0.5766,0.1080,0.492621,0.391118,0.432100,0.458614,0.449316,0.493759,0.491505,0.412240,0.429575,0.469598,0.461006,0.489766
1996-10-02,1.569398,3,0.4302,0.477875,0.478713,1996,10,2,5.0,6.0,5.0,5.0,5.0,5.0,0.0353,1.0692,1.2913,2.1190,0.8674,0.0653,0.492292,0.384300,0.450946,0.447974,0.438204,0.472939,0.491294,0.407437,0.442633,0.460626,0.451942,0.476704
1996-10-03,17.859299,2,0.4495,0.491203,0.490375,1996,10,3,4.0,6.0,5.0,5.0,5.0,5.0,0.2783,0.9147,1.2725,2.2432,1.2157,0.2746,0.496148,0.425672,0.435107,0.436272,0.501194,0.490613,0.493880,0.413252,0.438342,0.451673,0.478370,0.476392
1996-10-04,20.841913,2,0.3561,0.476827,0.483787,1996,10,4,4.0,6.0,5.0,5.0,5.0,5.0,0.6957,0.8338,1.3071,1.8349,1.4513,0.2632,0.484567,0.457851,0.419107,0.425025,0.483698,0.478728,0.485176,0.418785,0.430604,0.442405,0.491913,0.484252
1996-10-05,15.586549,1,0.5695,0.464411,0.472448,1996,10,5,4.0,6.0,5.0,5.0,5.0,5.0,0.6552,0.9839,1.4173,1.3773,1.6767,0.3715,0.473772,0.490829,0.406261,0.414356,0.468232,0.462643,0.479636,0.480346,0.422279,0.433058,0.478959,0.471617
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-11-26,0.000000,2,0.3398,0.342145,0.373197,2019,11,26,1.0,8.0,7.0,5.0,4.0,3.0,1.0919,1.3378,1.9191,2.5294,1.8076,1.0424,0.352499,0.450357,0.462952,0.460949,0.427921,0.391507,0.389813,0.461422,0.458716,0.453349,0.428227,0.411377
2019-11-27,0.000000,1,0.6084,0.493269,0.444673,2019,11,27,2.0,8.0,7.0,5.0,5.0,3.0,0.9114,1.4613,1.9520,2.6146,1.9560,0.9014,0.339058,0.427612,0.444662,0.472428,0.409340,0.439617,0.380560,0.446058,0.448385,0.455172,0.420137,0.433978
2019-11-28,0.951774,1,0.7808,0.470274,0.462430,2019,11,28,3.0,8.0,8.0,6.0,5.0,4.0,0.6524,1.7816,1.9104,2.2046,2.2855,1.0290,0.361567,0.405825,0.427252,0.466658,0.422816,0.420148,0.381075,0.430731,0.439063,0.458762,0.421203,0.428518
2019-11-29,2.152993,1,0.9184,0.444326,0.449410,2019,11,29,3.0,8.0,8.0,6.0,5.0,4.0,0.5886,1.3550,1.6640,2.1225,2.4898,0.9807,0.362398,0.385543,0.494899,0.456993,0.447623,0.413893,0.379400,0.415153,0.481285,0.451284,0.431803,0.425161
